In [50]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score


In [51]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [52]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [53]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [54]:
# Check the column names of the DataFrame
print(df_market_data.columns)


Index(['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'],
      dtype='object')


In [55]:
#Insert Coinid as an index
df_market_data.insert(0, 'coinid', ['bitcoin', 'ethereum', 'tether', 'ripple', 'bitcoin-cash',
'binancecoin','chainlink','cardano','litecoin','bitcoin-cash-sv','crypto-com-chain','usd-coin',
'eos','monero','tron','tezos','okb','stellar','cosmos','cdai','neo','wrapped-bitcoin','leo-token',
'huobi-token','nem','binance-usd','iota','vechain','zcash','theta-token','dash','ethereum-classic',
'ethlend','maker','havven','omisego','celsius-degree-token','ontology','ftx-token','true-usd','digibyte'])

In [56]:
from sklearn.preprocessing import StandardScaler

# Define the columns to be scaled
cols_to_scale = ["price_change_percentage_24h",
                 "price_change_percentage_7d",
                 "price_change_percentage_14d",
                 "price_change_percentage_30d", 
                 "price_change_percentage_60d", 
                 "price_change_percentage_200d", 
                 "price_change_percentage_1y"]

# Scale the data using StandardScaler
scaler = StandardScaler()
df_market_data_scaled = scaler.fit_transform(df_market_data[cols_to_scale])
df_market_data_scaled

array([[ 5.08529366e-01,  4.93193071e-01,  7.72200433e-01,
         2.35459633e-01, -6.74950963e-02, -3.55953481e-01,
        -2.51636882e-01],
       [ 1.85445894e-01,  9.34445040e-01,  5.58692121e-01,
        -5.43409317e-02, -2.73482725e-01, -1.15759474e-01,
        -1.99352110e-01],
       [ 2.17739616e-02, -7.06336853e-01, -2.16804207e-02,
        -6.10301536e-02,  8.00452481e-03, -5.50246924e-01,
        -2.82060506e-01],
       [-4.07643829e-02, -8.10928066e-01,  2.49457974e-01,
        -5.03879651e-02, -3.73164019e-01, -4.58258816e-01,
        -2.95546142e-01],
       [ 1.19303608e+00,  2.00095907e+00,  1.76061001e+00,
         5.45842065e-01, -2.91202870e-01, -4.99847761e-01,
        -2.70316950e-01],
       [ 8.91870708e-01,  1.32729453e+00,  8.00214184e-01,
        -5.71478992e-02,  7.78653106e-01, -1.88231917e-01,
        -2.25532605e-01],
       [ 1.13972400e-02,  2.57225091e+00,  1.10164693e+00,
        -4.90495415e-01, -9.31954023e-01,  3.87758986e-01,
        -1.8284399

In [57]:
# Create a DataFrame with the scaled data

market_data_transformed = pd.DataFrame(df_market_data_scaled, 
                                       columns=["price_change_percentage_24h",
                                                "price_change_percentage_7d",
                                                "price_change_percentage_14d",
                                                "price_change_percentage_30d", 
                                                "price_change_percentage_60d", 
                                                "price_change_percentage_200d", 
                                                "price_change_percentage_1y"])
market_data_transformed


# Display sample data
print(market_data_transformed.head())


   price_change_percentage_24h  price_change_percentage_7d  \
0                     0.508529                    0.493193   
1                     0.185446                    0.934445   
2                     0.021774                   -0.706337   
3                    -0.040764                   -0.810928   
4                     1.193036                    2.000959   

   price_change_percentage_14d  price_change_percentage_30d  \
0                     0.772200                     0.235460   
1                     0.558692                    -0.054341   
2                    -0.021680                    -0.061030   
3                     0.249458                    -0.050388   
4                     1.760610                     0.545842   

   price_change_percentage_60d  price_change_percentage_200d  \
0                    -0.067495                     -0.355953   
1                    -0.273483                     -0.115759   
2                     0.008005                     -0.55

---

In [58]:
# Copy the tickers names from the original data
market_data_transformed["coinid"] = df_market_data.index

# Set the Ticker column as index
market_data_transformed = df_market_data.set_index("coinid")
market_data_transformed

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coinid,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


### Find the Best Value for k Using the Original Data.

In [59]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [60]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for i in k:
# 1. Create a KMeans model using the loop counter for the n_clusters
    k_model = KMeans(n_clusters=i,random_state=1)
# 2. Fit the model to the data using `df_market_data_scaled`
    k_model.fit(df_market_data_scaled)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(k_model.inertia_)


In [61]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [62]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [63]:
# Initialize the K-Means model using the best value for k
k_model = KMeans(n_clusters=4, random_state=0)


In [64]:
# Fit the K-Means model using the scaled data
k_model.fit(df_market_data_scaled)


KMeans(n_clusters=4, random_state=0)

In [65]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predictions = k_model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(predictions)

[0 0 3 3 0 0 0 0 0 3 3 3 3 0 3 0 3 3 0 3 3 0 3 3 3 3 3 3 0 3 3 3 1 0 3 3 2
 3 3 3 3]


In [66]:
# Create a copy of the DataFrame
predictions.copy()

array([0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3, 0,
       3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 0, 3, 3, 2, 3, 3, 3, 3],
      dtype=int32)

In [67]:
# Add a new column to the DataFrame with the predicted clusters

market_data_transformed["predictions"] = predictions


# Display sample data
market_data_transformed.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predictions
coinid,,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761,0
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023,0
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954,3
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193,3
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384,0


In [68]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_data_transformed.hvplot.scatter(
                        x = "price_change_percentage_24h",
                        y = "price_change_percentage_7d",
                        c = "predictions",
                        hover_cols = "coinid"
)

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,predictions,coinid)

---

### Optimize Clusters with Principal Component Analysis.

In [69]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [70]:
# Use the PCA model with `fit_transform` to reduce to 
data_pca = pca.fit_transform(market_data_transformed)

# Instantiate the PCA instance and declare the number of PCA variable


# three principal components.

# View the first five rows of the DataFrame. 
data_pca[:5]



array([[-341.80076297,  -51.36736823,   12.52616664],
       [-249.42026714,   24.11695471,  -14.23076936],
       [-402.61482131, -118.71044146,   24.83804222],
       [-406.7525377 ,  -79.48699269,    1.56600748],
       [-382.42974792, -103.43255248,   16.75383434]])

In [71]:
# Retrieve the explained variance to determine how much information 
explained_var = pca.explained_variance_ratio_

# can be attributed to each principal component.
print('Explained variance for PC1: {:.2f}%'.format(explained_var[0]*100))
print('Explained variance for PC2: {:.2f}%'.format(explained_var[1]*100))
print('Explained variance for PC3: {:.2f}%'.format(explained_var[2]*100))



Explained variance for PC1: 97.60%
Explained variance for PC2: 2.30%
Explained variance for PC3: 0.07%


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 99.97%

In [72]:
# Create a new DataFrame with the PCA data.
df_data_pca = pd.DataFrame(
    data_pca,
    columns=["PCA1", "PCA2","PCA3"]
)


# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
df_data_pca["coinid"] = market_data_transformed.index

# Set the Ticker column as index
df_data_pca = df_data_pca.set_index("coinid")
df_data_pca



# Set the coinid column as index


# Display sample data
df_data_pca

,PCA1,PCA2,PCA3
coinid,,,
bitcoin,-341.800763,-51.367368,12.526167
ethereum,-249.420267,24.116955,-14.230769
tether,-402.614821,-118.710441,24.838042
ripple,-406.752538,-79.486993,1.566007
bitcoin-cash,-382.429748,-103.432552,16.753834
binancecoin,-289.124820,12.286577,34.164488
chainlink,28.151606,154.987404,-73.125761
cardano,-174.519633,80.242899,-30.392084
litecoin,-406.613142,-91.783622,5.016840


---

### Find the Best Value for k Using the PCA Data

In [73]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [74]:
# Create an empy list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
# 2. Fit the model to the data using `df_market_data_pca`
    k_model.fit(df_data_pca)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(k_model.inertia_)


In [75]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve

df_elbow = pd.DataFrame(elbow_data)
df_elbow

,k,inertia
0,1,6.997052e+07
1,2,8.180192e+06
2,3,2.580721e+06
3,4,8.237467e+05
4,5,4.264170e+05
5,6,2.592095e+05
6,7,1.897977e+05
7,8,1.472962e+05
8,9,1.086861e+05
9,10,8.556342e+04


In [76]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)


:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 2


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** yes the original was 3 but that was too broad for the clusters
  
  

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [77]:
df_data_pca.copy()
df_data_pca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, bitcoin to digibyte
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PCA1    41 non-null     float64
 1   PCA2    41 non-null     float64
 2   PCA3    41 non-null     float64
dtypes: float64(3)
memory usage: 1.3+ KB


In [78]:
# Initialize the K-Means model using the best value for k
k_means_model = KMeans(n_clusters=2, random_state = 0)

In [79]:
# Fit the K-Means model using the PCA data
k_means_model.fit(df_data_pca)


KMeans(n_clusters=2, random_state=0)

In [80]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters = k_model.predict(df_data_pca)
# Print the resulting array of cluster values.
clusters

array([4, 0, 4, 4, 4, 0, 5, 8, 4, 7, 8, 4, 4, 0, 0, 7, 7, 4, 0, 4, 0, 4,
       4, 4, 8, 4, 4, 8, 0, 3, 4, 4, 1, 4, 6, 9, 2, 4, 7, 4, 5],
      dtype=int32)

In [81]:
# Create a copy of the DataFrame with the PCA data
df_data_pca_copy = df_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
market_data_transformed['clusters'] = clusters
df_data_pca_copy['clusters'] = clusters

# Display sample data
df_data_pca_copy.head()


,PCA1,PCA2,PCA3,clusters
coinid,,,,
bitcoin,-341.800763,-51.367368,12.526167,4
ethereum,-249.420267,24.116955,-14.230769,0
tether,-402.614821,-118.710441,24.838042,4
ripple,-406.752538,-79.486993,1.566007,4
bitcoin-cash,-382.429748,-103.432552,16.753834,4


In [82]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_data_transformed.hvplot.scatter(
                        x = "price_change_percentage_24h",
                        y = "price_change_percentage_7d",
                        c = "clusters",
                        hover_cols = "coinid"
)

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,clusters,coinid)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [83]:
# Composite plot to contrast the Elbow curves
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

k_values = range(1, 11)
sse_values = []
silhouette_scores = []

for k in k_values:
    # Fit KMeans model with k clusters
    k_model = KMeans(n_clusters=k, random_state=42)
    k_model.fit(df_data_pca)
    
    # Append SSE and silhouette score to lists
    sse_values.append(k_model.inertia_)
    silhouette_scores.append(silhouette_score(df_data_pca, k_model.labels_))

# Plot Elbow curve
plt.plot(k_values, sse_values)
plt.xlabel('Number of clusters (k)')
plt.ylabel('Sum of Squared Error (SSE)')
plt.title('Elbow Curve')
plt.show()

# Plot Silhouette scores
plt.plot(k_values, silhouette_scores)
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette score')
plt.title('Silhouette Score')
plt.show()


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [84]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 